In [85]:
import torch
import torch.nn.functional as F
from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from torch import nn
import pandas as pd
import pyarrow as pa
from torch.optim import Adam
import numpy as np
import os

In [86]:
current_dir = os.getcwd()

In [87]:
label_files = os.path.join(current_dir, 'Label_Files')
vector_files = os.path.join(current_dir, 'Vector_files')

label_list = []
vector_list =[]

for filename in os.listdir(label_files):
    file_path = os.path.join(label_files, filename)
    label_list.append(torch.load(file_path))
    
for filename in os.listdir(vector_files):
    file_path = os.path.join(vector_files, filename)
    vector_list.append(torch.load(file_path))

In [88]:
labels = torch.cat(label_list).float()
# Divide all the labels by 4 
labels /= 4
final_pool = torch.cat(vector_list)

In [89]:
# convert our data into a torch dataset
#need y data from the file
ratings = torch.tensor(labels).float()

x_train, x_test, y_train, y_test = train_test_split(final_pool, ratings, test_size=0.15)

data = torch.utils.data.TensorDataset(torch.tensor(x_train).float(), torch.tensor(y_train).float())

# set up a data loader to get batches from our dataset
loader = torch.utils.data.DataLoader(data, batch_size = 16, shuffle = True)

/var/folders/45/vvds_7n55h151_y61fjd1fmc0000gn/T/ipykernel_86459/2224312218.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  ratings = torch.tensor(labels).float()
/var/folders/45/vvds_7n55h151_y61fjd1fmc0000gn/T/ipykernel_86459/2224312218.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data = torch.utils.data.TensorDataset(torch.tensor(x_train).float(), torch.tensor(y_train).float())


In [90]:
model = torch.nn.Sequential(   
    torch.nn.Linear(768,64),
    torch.nn.Sigmoid(),
    torch.nn.Linear(64,16),
    #torch.nn.Sigmoid(),
    torch.nn.Linear(16,1)
    # torch.nn.Sigmoid(),
    # torch.nn.Linear(8,4),
    # torch.nn.Softmax(dim=1)
)
# Figure out whether its coming from sigmoid, loss function, or something else 

In [94]:
scoring_function = torch.nn.MSELoss()

num_epochs = 10

optimizer = torch.optim.SGD( model.parameters(), .001 )

In [97]:
def train():
    
    for epoch in range(num_epochs):

        for batch in loader:
            # prepare the input and output tensors for the current batch
            batchX = torch.tensor(batch[0], dtype=torch.float32)
            batchY = torch.tensor(batch[1], dtype=torch.float32)
            #batchY = batchY.clone().detach().requires_grad_(True)
            batchY = batchY.unsqueeze(1)
            #print(batchX)
            # forward pass
            y_pred = model.forward(batchX)

            
            #print(y_pred)
            #print(batchY)
            #print(y_pred)
            #print(y_pred)
            #print(batchY)
            
            #y_pred.requires_grad = True
            

            # Compute the loss
            loss = scoring_function(y_pred, batchY)
            
            # Compute the gradient of the error with respect to the model parameters
            optimizer.zero_grad()
            loss.backward()
        
            # update the model parameters
            optimizer.step()
        
        # print the epoch and the average error for the epoch
        print('Epoch {}, Average Error: {}'.format(epoch, loss.mean()))
        
            

In [98]:
train()

/var/folders/45/vvds_7n55h151_y61fjd1fmc0000gn/T/ipykernel_86459/2600830434.py:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batchX = torch.tensor(batch[0], dtype=torch.float32)
/var/folders/45/vvds_7n55h151_y61fjd1fmc0000gn/T/ipykernel_86459/2600830434.py:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  batchY = torch.tensor(batch[1], dtype=torch.float32)


Epoch 0, Average Error: 0.06371079385280609
Epoch 1, Average Error: 0.06283369660377502
Epoch 2, Average Error: 0.1834488809108734
Epoch 3, Average Error: 0.06918289512395859
Epoch 4, Average Error: 0.07512538135051727
Epoch 5, Average Error: 0.04353857785463333
Epoch 6, Average Error: 0.054870761930942535
Epoch 7, Average Error: 0.09319761395454407
Epoch 8, Average Error: 0.06391926109790802
Epoch 9, Average Error: 0.012305828742682934
